# Dekoratory

In [37]:
import functools

def shouter(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print('Before call ---- ', func.__name__)
        result = func(*args, **kwargs)
        print('After call ---- ', func.__name__)
        return result
    
    wrapper.__name__ = wrapper.__name__ + "_decorated"
    return wrapper

In [40]:
@shouter
def foo():
    """foo functions"""
    print('foo()')
    return "foo_42"

In [41]:
foo.__name__

'foo_decorated'

In [33]:
foo.__doc__

'foo functions'

In [19]:
foo()

Before call ----  wrapper
Before call ----  foo
foo()
After call ----  foo
After call ----  wrapper


'foo_42'

In [20]:
foo = shouter(foo)

In [23]:
from typing import Callable
from typing import Any


def disable(func: Callable[..., Any]) -> Callable[..., Any | None]:
    @functools.wraps(func)
    def _disable(*args, **kwargs):
        pass
    return _disable

@shouter
def bar(n):
    print(f"bar({n})")

In [24]:
bar(13)

# Dekoratory z parametrami

In [44]:
def tag(tagname):
    def tag_decorator(func):
        @functools.wraps(func)
        def _tag_decorator(*args, **kwargs):
            tag_before = f"<{tagname}>"
            tag_after = f"</{tagname}>"
            return tag_before + func(*args, **kwargs) + tag_after
        return _tag_decorator
    return tag_decorator        

@tag("h1")
@tag("b")
def output(data):
    return data

In [45]:
output("Text")

'<h1><b>Text</b></h1>'

# Rejestracja za pomocą dekoratorów

In [64]:
import collections


class EventRegistry:
    def __init__(self) -> None:
        self.registry = collections.defaultdict(list)

    def on(self, *events):
        def _on(function: Callable) -> Callable:
            for event in events:
                self.registry[event].append(function)
            return function
        return _on
    
    def fire(self, event: str, *args, **kwargs):
        for function in self.registry[event]:
            function(*args, **kwargs)

In [79]:
events = EventRegistry()

@events.on('start', 'success')
def start(service_name: str):
    print(f'Starting service {service_name}')

@events.on('error')
def teardown():
    print('Shutting down system')

class Printer:
    def __init__(self, id: int):
        self.id = id
    
    def run(self, service_name: str):
        print(f'Running printer: {self.id}, {service_name}')

printer = Printer(42)

bound_run = functools.partial(Printer.run, printer)

bound_run = events.on('start')(bound_run) # dynamic registration

In [80]:
events.fire('start', "Deamon665")

Starting service Deamon665
Running printer: 42, Deamon665


# Tworzenie dekoratorów za pomocą klas

In [86]:
class Debug:
    def __init__(self, function: Callable) -> None:
        self.function = function
        functools.update_wrapper(self, function)

    def __call__(self, *args, **kwargs):
        result = self.function(*args, **kwargs)
        name = self.function.__name__

        print(f"{name}({args!r}, {kwargs!r}): {result}")

In [87]:
@Debug
def add(a, b):
    return a + b

In [88]:
add(1, 3)

add((1, 3), {}): 4


# Properties

In [95]:
class IntProperty:
    def __init__(self, fget=None, fset=None, fdel=None) -> None:
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
    
    def __get__(self, obj, owner_cls):
        if obj is None:
            return self
        elif self.fget:
            return self.fget(obj)

    def __set__(self, obj, value):
        if not isinstance(value, int):
            raise ValueError("Value must be int")
        self.fset(obj, value)

    def __delete__(self, obj):
        self.fdel(obj)

    def getter(self, fget):
        return IntProperty(fget, self.fset, self.fdel)

    def setter(self, fset):
        return IntProperty(self.fget, fset, self.fdel)
    
    def deleter(self, fdel):
        return IntProperty(self.fget, self.fset, fdel)

In [96]:
class Person:

    def __init__(self, age) -> None:
        self.__age = age

    @IntProperty
    def age(self):
        return self.__age
    
    @age.setter
    def age(self, new_age):
        self.__age = new_age



In [97]:
p = Person(42)

In [98]:
p.age

42

In [99]:
p.age = 33

In [100]:
p.age

33

In [101]:
p.age = '42'

ValueError: Value must be int

# Przykłady 

## Counter

In [119]:
def counter(function: Callable[..., Any]) -> Callable[..., Any]:
    function.calls = 0
    
    @functools.wraps(function)
    def _counter(*args, **kwargs):
        function.calls += 1
        return function(*args, **kwargs)
    
    return _counter

In [121]:
@functools.lru_cache()
@counter
def fibonacci(n: int) -> int:
    if n < 2:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [122]:
fibonacci(20)

6765

In [124]:
fibonacci.__wrapped__.__wrapped__.calls

21

## Sort by attribute

In [128]:
def sort_by_attribute(attr, keyfunc=getattr):
    def _sort_by_attribute(cls):
        def __lt__(self, other):
            return keyfunc(self, attr) < keyfunc(other, attr)
        
        def __eq__(self, other):
            return keyfunc(self, attr) == keyfunc(other, attr)
        
        # injecting special function to the class
        cls.__lt__ = __lt__
        cls.__eq__ = __eq__

        return functools.total_ordering(cls)
    
    return _sort_by_attribute


In [135]:
@sort_by_attribute('name')
class Gadget:
    def __init__(self, id, name):
        self.id = id
        self.name = name
    
    def __repr__(self) -> str:
        return f'<{self.__class__.__name__} {self.id} {self.name}>'

In [136]:
gadgets = [Gadget(1, "ipad"), Gadget(5, "smartwatch"), Gadget(3, "a gadget"), Gadget(665, "laptop")]

sorted(gadgets)

[<Gadget 3 a gadget>,
 <Gadget 1 ipad>,
 <Gadget 665 laptop>,
 <Gadget 5 smartwatch>]

## Synchronized

In [141]:
from threading import Lock

def synchronized(lock):
    """Synchronized decorator"""
    def decorate(function):
        @functools.wraps(function)
        def _synchronized(*args, **kwargs):
            with lock:
                return function(*args, **kwargs)
        return _synchronized
    return decorate

In [142]:
from threading import Lock

class MyLock:
    def acquire(self):
        print(f"{self}.acquire()")

    def release(self):
        print(f"{self}.release()")

    def __enter__(self):
        self.acquire()
        return self

    def __exit__(self, excpt_type, excpt_val, excpt_tb):
        self.release()

my_lock = MyLock()

@synchronized(my_lock)
def critical_section_1():
    print(critical_section_1.__name__)

In [143]:
critical_section_1()

<__main__.MyLock object at 0x000001EDFE20E890>.acquire()
critical_section_1
<__main__.MyLock object at 0x000001EDFE20E890>.release()


## AddId

In [156]:
def add_id(decorated_class):
    setattr(decorated_class, '__id_gen', 0)
    print(f'Adding __id_gen to class {decorated_class.__name__}')
    original_init = decorated_class.__init__

    def __init__(self, *args, **kwargs):
        decorated_class.__id_gen += 1
        self.id = decorated_class.__id_gen
        print(f'Adding id={self.id} to object {self}')
        original_init(self, *args, **kwargs)

    decorated_class.__init__ = __init__
    return decorated_class

In [157]:
@add_id
class Person:
    def __init__(self, name):
        self.name = name

Adding __id_gen to class Person


In [158]:
p1 = Person("John")

Adding id=1 to object <__main__.Person object at 0x000001EDFE8D7990>


In [159]:
Person.__id_gen

1

In [160]:
p1.id

1

In [161]:
p2 = Person("Ewa")

Adding id=2 to object <__main__.Person object at 0x000001EDFE8ACA10>


In [162]:
p2.id

2

In [163]:
class Manager(Person):
    def __init__(self, name, rank):
        super().__init__(name)
        self.rank = rank

In [164]:
m1 = Manager("Jim", 10)

Adding id=3 to object <__main__.Manager object at 0x000001EDFE80E310>


In [165]:
m2 = Manager("Ola", 10)

Adding id=4 to object <__main__.Manager object at 0x000001EDFE866290>


# Singelton

In [173]:
import functools

def singleton(cls):
    instances = dict()

    @functools.wraps(cls)
    def _singleton(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return _singleton


@singleton
class Logger:
    def __init__(self):
        print(f'Executing Logger.__init__({self})')
    
    def log(self, msg: str) -> None:
        print(f">>{msg}")

In [174]:
log1 = Logger()

Executing Logger.__init__(<__main__.Logger object at 0x000001EDFE7EAC90>)


In [175]:
log2 = Logger()

In [176]:
log1 is log2

True

In [180]:
class Singleton(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Logger(metaclass=Singleton):
    def __init__(self):
        print(f'Executing Logger.__init__({self})')
    
    def log(self, msg: str) -> None:
        print(f">>{msg}")

In [181]:
class BetterLogger(Logger):
    def __init__(self):
        print(f'Executing BetteLogger.__init__({self})')

In [182]:
log1 = Logger()

Executing Logger.__init__(<__main__.Logger object at 0x000001EDFEA8D850>)


In [183]:
log2 = Logger()

In [184]:
log1 is log2

True

In [185]:
blog1 = BetterLogger()

Executing BetteLogger.__init__(<__main__.BetterLogger object at 0x000001EDFE1F71D0>)


In [186]:
blog2 = BetterLogger()

In [187]:
blog1 is blog2

True